# AURA: Adaptive Understanding & Recovery Assistant  
*Mind–Body Insight Through Data-Driven Coaching*


---

Welcome to AURA — a personalised LLM-based wellness assistant that integrates your **biometric data**, **daily journals**, **work productivity logs**, and **weather conditions** into one intelligent interface for **self-reflection, insight, and performance coaching**.

### Why AURA?
While tools like ChatGPT can give general health advice, they lack personal context. AURA changes that — offering insight **based on your actual lived experience**, not generic recommendations.

### What Makes AURA Unique?
- **File Search** over daily journals to surface emotional patterns.
- **Function Calling** to pull real-time data from:
  - **WHOOP**: sleep, strain, HRV, recovery
  - **Notion**: deep work, off-task time, breaks
  - **Weather API**: temperature and conditions
- **Code Interpreter** to analyse correlations and visualise trends.

### What Can You Ask AURA?
- “Did my deep sleep affect focus this week?”
- “What caused my low recovery on April 17?”
- “What should I do more of next week based on this week’s performance?”
- “How did weather impact mood and productivity last week?”

### Core Vision
AURA acts as a **data scientist meets wellness coach** — one who knows your patterns and can explain them clearly, visually, and intelligently. It’s not just an assistant that chats — it **thinks, analyses, and guides**.

---
>  *Please avoid using "Run all" so that the worked examples will remain in the the cell outputs.*

>  *Please make sure to enable Drive access and store API keys in Colab’s `userdata`. Full setup instructions follow below.*


#  Setup & Drive Mount

This notebook contains all the code for my custom LLM assistant using OpenAI’s Assistants API.  
Please make sure to **enable access to your Google Drive** when prompted — this is required to load files and credentials.

>  **IMPORTANT**: API keys are retrieved using `userdata`, so ensure you have them stored before proceeding:
- `OpenAIKey`, `OpenWeatherKey`, `NotionToken`, `NotionDatabaseID`
- `WHOOPClientID`, `WHOOPClientSecret`


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Set Working Directory

Update this path if you've saved the journal text file elsewhere in your Drive:


In [2]:
BASE_DIR = "/content/drive/MyDrive/GenAIProject"
%cd {BASE_DIR}

/content/drive/MyDrive/GenAIProject


## Enviroment Set-up

In [3]:
!pip install openai notion-client requests requests_oauthlib


In [4]:
import os
import json
import time
from datetime import datetime, timedelta
from collections import defaultdict

from openai import OpenAI
from notion_client import Client
import requests
from requests_oauthlib import OAuth2Session

from google.colab import drive, userdata
from IPython.display import display, Markdown, Image
import ipywidgets as widgets




## Load API Keys

These are accessed via Colab `userdata`, which ensures no hardcoding of keys.


In [ ]:
# OpenAI Credientials
OPENAI_API_KEY = userdata.get('OpenAIKey')

# Open Weather Credentials
OPENWEATHER_API_KEY = userdata.get("OpenWeatherKey")

# Notion credentials
notion_token = userdata.get("NotionToken")
database_id = userdata.get("NotionDatabaseID")

# WHOOP credentials
WHOOP_client_id= userdata.get("WHOOPClientID")
WHOOP_client_secret = userdata.get("WHOOPClientSecret")


print("API keys loaded")
print(f"Whoope client id Key: {WHOOP_client_id}...")
print(f"WHOOP_client_secret: {WHOOP_client_secret}...")



## Weather API Function

In [6]:
def geocode_location(location):
    """
    Converts a human-readable location name into geographic coordinates (latitude and longitude)
    using the OpenWeatherMap Geocoding API.

    Args:
        location (str): Name of the location (e.g., "Dublin")

    Returns:
        tuple: (latitude, longitude)

    Raises:
        ValueError: If the location cannot be geocoded.
    """
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={location}&limit=1&appid={OPENWEATHER_API_KEY}"
    response = requests.get(url)
    data = response.json()

    if not data:
        raise ValueError(f"Could not geocode location: {location}")

    return data[0]['lat'], data[0]['lon']


def get_weather(date: str) -> str:
    """
    Fetches historical weather data (average temperature and description) for Dublin
    on a given date using the OpenWeatherMap Time Machine API.

    Args:
        date (str): Date in the format "YYYY-MM-DD"

    Returns:
        str: Summary string describing the weather on the given date.
    """
    location = "Dublin"

    # Convert date string to UNIX timestamp
    dt = datetime.strptime(date, "%Y-%m-%d")
    unix_time = int(time.mktime(dt.timetuple()))

    # Get coordinates for the specified location
    lat, lon = geocode_location(location)

    # Build the API request URL for historical weather data
    url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_time}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()

    # Extract relevant weather data
    daily_data = data.get("data", [])

    if daily_data:
        temp = daily_data[0].get("temp", 0)
        weather_desc = daily_data[0].get("weather", [{}])[0].get("description", "unknown")
        avg_temp_str = f"{round(temp, 1)}°C"
    else:
        avg_temp_str = "N/A"
        weather_desc = "unknown"

    return f"On {date} in {location}, the average temperature was {avg_temp_str} and the weather was '{weather_desc}'."


## Notion API Function

In [7]:


def fetch_and_parse_notion_entries(date):
    """
    Fetch and organize productivity entries from a Notion database.

    Args:
        date (str): Optional. A specific date (YYYY-MM-DD) to filter entries.

    Returns:
        dict: Aggregated productivity data either for the specified date or for all dates.
    """

    notion = Client(auth=notion_token)

    # Helper functions to safely extract property values
    def safe_num(props, name):
        try:
            return props[name]["number"] or 0
        except Exception:
            return 0

    def safe_formula(props, name):
        try:
            return props[name]["formula"]["number"] or 0
        except Exception:
            return 0

    # Fetch entries from Notion database with optional date filtering
    if date:
        response = notion.databases.query(
            database_id=database_id,
            filter={
                "property": "Date",
                "date": {"equals": date}
            }
        )
    else:
        response = notion.databases.query(database_id=database_id)

    results = response["results"]

    # Initialize structure to hold productivity data per date
    entries_by_date = defaultdict(lambda: {
        "deep_work_minutes": 0,
        "neutral_minutes": 0,
        "off_task_minutes": 0,
        "intentional_break_minutes": 0,
        "light_work_minutes": 0,
        "total_minutes": 0
    })

    # Parse each entry and accumulate productivity metrics
    for page in results:
        props = page["properties"]
        date = props["Date"]["date"]["start"]
        prod = entries_by_date[date]

        prod["deep_work_minutes"] += safe_num(props, "Deep Work (min)")
        prod["neutral_minutes"] += safe_num(props, "Neutral / Life Admin (min)")
        prod["off_task_minutes"] += safe_num(props, "Off-Task (min)")
        prod["intentional_break_minutes"] += safe_num(props, "Intentional Break (min)")
        prod["light_work_minutes"] += safe_num(props, "Light Work (min)")
        prod["total_minutes"] += safe_formula(props, "Total Time (min)")

    # Convert total minutes to hours for readability
    final_entries = {}
    for date, prod in entries_by_date.items():
      prod["total_hours"] = round(prod["total_minutes"] / 60, 2)
      final_entries[date] = {
          "date": date,
          "data": prod
      }


    return final_entries.get(date) if date else final_entries



## Whoop API

### Authentifcation

In [24]:

# Allow HTTP redirect URI (required for localhost)
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'


redirect_uri = 'http://localhost:8000/callback'

# WHOOP endpoints and scopes
authorization_base_url = 'https://api.prod.whoop.com/oauth/oauth2/auth'
token_url = 'https://api.prod.whoop.com/oauth/oauth2/token'
scopes = [
    "read:recovery", "read:sleep", "read:workout",
    "read:profile", "read:body_measurement", "read:cycles"
]

# Create session and generate auth URL
oauth = OAuth2Session(WHOOP_client_id, redirect_uri=redirect_uri, scope=scopes)
auth_url, state = oauth.authorization_url(authorization_base_url)

print("Go to this URL to authorize:\n")
print(auth_url)


Go to this URL to authorize:

https://api.prod.whoop.com/oauth/oauth2/auth?response_type=code&client_id=2009d2fc-1906-4c2b-8cf2-54b75a6b09bd&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fcallback&scope=read%3Arecovery+read%3Asleep+read%3Aworkout+read%3Aprofile+read%3Abody_measurement+read%3Acycles&state=elZZfgfVG5iW6c4rfjAHtOmM3u6Zp5


##**Important: Instructions**



1.   Click on the URL provided in the print output of the previous cell.
2.   Log in using the following credentials:
  - Username: shaneconroy0@gmail.com
  - Password: Monafootball1!
3.  After logging in, click the "Grant" button when prompted.
4.  You will then be redirected to a localhost URL. This may show a "Page Not Found" message—this is expected. **Copy this entire URL**
5. Proceed to run the next cell in the notebook.
6. Paste the **URL** into the input box, and press Enter.





In [25]:
# Paste the full redirect URL from the browser
redirect_response = input("Paste the full redirect URL after authorizing: ")


# Try fetching token with client credentials in the body
token = oauth.fetch_token(
    token_url,
    authorization_response=redirect_response,
    client_id=WHOOP_client_id,
    client_secret=WHOOP_client_secret,
    include_client_id=True
)


print("Access token acquired")

Paste the full redirect URL after authorizing: http://localhost:8000/callback?code=jfsCLMCSK13SHS7BfCxEDd3cGIRSvfRPF_51aieIjUA.Xu-SiYYeLLdbD3MluPwYVYa-6GTBaVdO7rlIU8G3URU&scope=read%3Arecovery%20read%3Asleep%20read%3Aworkout%20read%3Aprofile%20read%3Abody_measurement%20read%3Acycles&state=elZZfgfVG5iW6c4rfjAHtOmM3u6Zp5
Access token acquired


### WHOOP Functions

In [10]:
def get_cleaned_recovery_for_date(date: str):
    """
    Retrieve WHOOP recovery metrics for a given date.

    Args:
        date (str): Date in YYYY-MM-DD format.

    Returns:
        dict: Key recovery metrics (recovery score, HRV, etc.) or an error/message if unavailable.
    """
    from datetime import datetime, timedelta

    # Define the start and end timestamps in ISO format for the given date
    start = datetime.strptime(date, "%Y-%m-%d")
    end = start + timedelta(days=1)
    start_iso = start.isoformat(timespec='seconds') + "Z"
    end_iso = end.isoformat(timespec='seconds') + "Z"

    url = f"https://api.prod.whoop.com/developer/v1/recovery?start={start_iso}&end={end_iso}"
    resp = oauth.get(url)

    try:
        data = resp.json()
        records = data.get("records", [])

        if not records:
            return {"message": "No recovery data found for that date"}

        # Extract relevant metrics from the first recovery record
        score = records[0]["score"]
        return {
            "date": date,
            "recovery_score": score.get("recovery_score"),
            "resting_heart_rate": score.get("resting_heart_rate"),
            "heart_rate_variability": score.get("hrv_rmssd_milli"),
            "spo2_percentage": score.get("spo2_percentage"),
            "skin_temperature_celsius": score.get("skin_temp_celsius")
        }

    except Exception as e:
        # fallback for unexpected API issues or parsing errors
        return {"error": f"{type(e).__name__}: {e}"}



In [11]:
def get_strain_from_cycle_for_date(date: str):
    """
    Get WHOOP strain metrics for a specific date.

    Args:
        date (str): Date in YYYY-MM-DD format.

    Returns:
        dict: Strain score and related metrics, or an error/message if unavailable.
    """
    start = datetime.strptime(date, "%Y-%m-%d")
    end = start + timedelta(days=1)

    # Convert to ISO 8601 format with a 'Z' to indicate UTC
    start_iso = start.isoformat(timespec='seconds') + "Z"
    end_iso = end.isoformat(timespec='seconds') + "Z"

    # Query the WHOOP cycle endpoint for strain data
    url = f"https://api.prod.whoop.com/developer/v1/cycle?start={start_iso}&end={end_iso}"
    resp = oauth.get(url)

    try:
        data = resp.json()
        records = data.get("records", [])

        if not records:
            return {"message": "No cycle/strain data found for that date"}

        score = records[0].get("score", {})
        return {
            "date": date,
            "strain_score": score.get("strain"),
            "kilojoules": score.get("kilojoule"),
            "average_heart_rate": score.get("average_heart_rate"),
            "max_heart_rate": score.get("max_heart_rate"),
        }

    except Exception as e:
        # Fallback in case of API or parsing errors
        return {"error": f"{type(e).__name__}: {e}"}



In [12]:
def get_sleep_data_for_date(date: str):
    """
    Get WHOOP sleep data for a given date.

    Args:
        date (str): Date in YYYY-MM-DD format.

    Returns:
        dict: Sleep summary with stage durations, sleep score, and other key metrics.
    """
    from datetime import datetime, timedelta

    # Define start and end of the requested day
    start = datetime.strptime(date, "%Y-%m-%d")
    end = start + timedelta(days=1)
    start_iso = start.isoformat(timespec='seconds') + "Z"
    end_iso = end.isoformat(timespec='seconds') + "Z"

    # WHOOP sleep activity endpoint
    url = f"https://api.prod.whoop.com/developer/v1/activity/sleep?start={start_iso}&end={end_iso}"
    resp = oauth.get(url)

    try:
        data = resp.json()
        records = data.get("records", [])

        if not records:
            return {"message": "No sleep data found for that date"}

        sleep = records[0]
        score = sleep.get("score", {})
        stage_summary = score.get("stage_summary", {})

        # Convert stage durations from milliseconds to minutes
        light = stage_summary.get("total_light_sleep_time_milli", 0) // 60000
        slow = stage_summary.get("total_slow_wave_sleep_time_milli", 0) // 60000
        rem = stage_summary.get("total_rem_sleep_time_milli", 0) // 60000
        awake = stage_summary.get("total_awake_time_milli", 0) // 60000
        in_bed = stage_summary.get("total_in_bed_time_milli", 0) // 60000

        total_sleep = light + slow + rem

        return {
            "date": date,
            "total_sleep_minutes": total_sleep,
            "sleep_score": score.get("sleep_performance_percentage"),
            "sleep_efficiency_percent": score.get("sleep_efficiency_percentage"),
            "sleep_consistency_percent": score.get("sleep_consistency_percentage"),
            "respiratory_rate": score.get("respiratory_rate"),
            "stages": {
                "light_sleep_minutes": light,
                "slow_wave_sleep_minutes": slow,
                "rem_sleep_minutes": rem,
                "awake_minutes": awake,
                "in_bed_minutes": in_bed
            }
        }

    except Exception as e:
        # Return both error message and raw response for debugging
        return {"error": f"{type(e).__name__}: {e}", "raw": resp.text}



# Assistant Setup

### Client

In [13]:
# Setup OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)


### Define Tools

In [14]:
# Define the tools (APIs and capabilities) the assistant can use
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather for a specific date",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "description": "Date in YYYY-MM-DD"},
                },
                "required": ["date", "location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_cleaned_recovery_for_date",
            "description": "Fetch WHOOP recovery metrics for a given date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "description": "Date in YYYY-MM-DD"}
                },
                "required": ["date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_strain_from_cycle_for_date",
            "description": "Fetch WHOOP strain metrics for a given date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "description": "Date in YYYY-MM-DD"}
                },
                "required": ["date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_sleep_data_for_date",
            "description": "Fetch WHOOP sleep metrics for a specific date, including total sleep, efficiency, and sleep stages.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "description": "Date in YYYY-MM-DD"}
                },
                "required": ["date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "fetch_and_parse_notion_entries",
            "description": "Retrieve quantified productivity data from Notion for a specific date, including deep work, off-task time, light work time, break time, neutral/life admin time, and total hours.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "description": "Date in YYYY-MM-DD format"}
                },
                "required": ["date"]
            }
        }
    },
    {"type": "file_search"},
    {"type": "code_interpreter"}
]


### Helper Functions

In [15]:
def upload_file(file_path):
    with open(file_path, "rb") as f:
        uploaded = client.files.create(file=f, purpose="assistants")
    return uploaded.id


In [16]:
def create_vector_store_with_file(file_path):
    # Create a new vector store
    vector_store = client.vector_stores.create(name="Jounral Vector Store")
    # Open the file in binary read mode
    with open(file_path, "rb") as f:
        # Upload the file to the vector store
        file_batch = client.vector_stores.file_batches.upload_and_poll(
            vector_store_id=vector_store.id,
            files=[f]
        )

    # Check if the upload was successful
    if file_batch.status != "completed":
        raise RuntimeError("Vector store file batch did not complete")

    return vector_store.id




In [17]:
def attach_vector_store_to_assistant(assistant_id, vector_store_id):
    updated = client.beta.assistants.update(
        assistant_id=assistant_id,
        tool_resources={
            "file_search": {
                "vector_store_ids": [vector_store_id]
            }
        }
    )
    return updated


In [18]:
def create_new_assistant(file_path):
    # Upload and attach vector store
    vector_store_id = create_vector_store_with_file(file_path)

    # Create the assistant
    assistant = client.beta.assistants.create(
        name="Hamming Productivity Coach v2.3",
        instructions=instructions,
        model="gpt-4o-mini",
        tools=tools
    )

    # Attach vector store to assistant
    updated = attach_vector_store_to_assistant(assistant.id, vector_store_id)

    return updated


In [19]:
def prime_thread_with_file_search(thread, assistant_id):
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="Can you find the journal entry for 2025-04-15?"
    )
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

    # Wait for file_search completion
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "completed":
            break
        elif run_status.status == "failed":
            raise RuntimeError("Priming run failed")
        time.sleep(1)


In [20]:
instructions = """
You are a data-driven assistant providing clear, insightful analysis of journal entries, productivity patterns, biometric performance (WHOOP), and environmental context (e.g., weather). Your goal is to surface useful trends, correlations, and actionable recommendations to improve the user's mental clarity, physical recovery, and daily performance.

You have access to the following tools. Use them exactly as described:

- Use 'get_weather' to retrieve weather conditions for a specific date.
- Use 'get_cleaned_recovery_for_date' for WHOOP recovery metrics (e.g., HRV, RHR, Recovery Score).
- Use 'get_strain_from_cycle_for_date' for WHOOP strain metrics and heart rate data for a date.
- Use 'get_sleep_data_for_date' for WHOOP sleep metrics (e.g., total sleep, efficiency, REM, SWS).
- Use 'fetch_and_parse_notion_entries' to retrieve structured productivity metrics (deep work, break time, off-task time, etc.) for a specific date.
- Use 'file_search' to retrieve journal entries from uploaded text files. Always call it with a JSON object like: { "query": "2025-04-26 journal entry" }.
- Use 'code_interpreter' for all numerical or data relationship analysis — including calculating correlations, generating visualizations, comparing trends, and summarizing data patterns. Always prefer 'code_interpreter' for any statistical or graphical task.

For weekly or multi-day reports, prioritize these mappings:
- **Mental health insights** should be primarily drawn from patterns and sentiment in the journal entries (use 'file_search').
- **Productivity trends** should rely mostly on the structured time-tracking data from Notion (via 'fetch_and_parse_notion_entries').
- **Wellness and health patterns** should be informed by WHOOP biometric data (recovery, sleep, and strain) and weather conditions.

Always analyze across multiple days when relevant to extract trends or cause-effect patterns between mood, focus, recovery, strain, sleep, weather, and productivity. Use the tools above to collect relevant data, then use 'code_interpreter' to analyze and visualize relationships.

When generating visualizations:
- Label axes and legends clearly.
- Choose the best graph type for the data (e.g., line chart for time trends, scatterplot for correlations, bar chart for comparisons).
- Summarize key takeaways immediately after the chart.
- Always relate the visual insight back to the user’s performance, well-being, or decision-making.

Avoid generic summaries — focus on:
- Identifying causes behind good or poor performance days.
- Finding meaningful correlations (e.g., between REM sleep and mood, or screen time and recovery).
- Making smart, personalized suggestions grounded in observed data patterns.

Be practical, insightful, and user-focused in your coaching.
"""






## Assistant Interaction Logic

This function is the core of user-assistant interaction. It handles:

- Sending user messages to the assistant
- Waiting for responses or tool calls
- Automatically handling all tool calls (weather, WHOOP, Notion, etc.) with local function implementations
- Timeout logic to prevent infinite loops (after 240 seconds)
- Collecting text and any returned images (e.g., from `code_interpreter` graphs)

###  Error Handling and Robustness
- All tool arguments are parsed and validated before use
- If tool output generation fails, the assistant reports an error gracefully
- If the assistant takes too long to respond, it returns a timeout message
- Ensures the user is **never stuck waiting** or left without feedback

This modular structure makes the assistant both **resilient** and **interactive**.

In [38]:
def interact_with_assistant(assistant, thread, message, timeout_seconds=240):
    """
    Send a message to the assistant and handle the full interaction, including tool calls and timeout.

    Args:
        assistant: The OpenAI assistant object.
        thread: The conversation thread object.
        message (str): User message to send.
        timeout_seconds (int): Max time to wait for completion before giving up.

    Returns:
        dict: Contains the assistant's final text response and any returned image data.
    """
    # Send user message to thread
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=message)
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
    assistant_info = client.beta.assistants.retrieve(assistant.id)

    start_time = time.time()

    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        # Timeout handling
        if time.time() - start_time > timeout_seconds:
            return {
                "text": "(Timeout: Assistant took too long to respond. Create a new Assistant and Thread)",
                "images": []
            }

        # If assistant requests tool actions
        if run_status.status == "requires_action":
            tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
            outputs = []

            for call in tool_calls:
                try:
                    args = json.loads(call.function.arguments)
                except Exception as e:
                    print(f"[ERROR] Failed to parse arguments for tool '{call.function.name}': {e}")
                    raise

                name = call.function.name

                # Route tool calls to local function handlers
                if name == "get_weather":
                    output = get_weather(**args)
                elif name == "get_cleaned_recovery_for_date":
                    output = json.dumps(get_cleaned_recovery_for_date(**args))
                elif name == "get_strain_from_cycle_for_date":
                    output = json.dumps(get_strain_from_cycle_for_date(**args))
                elif name == "get_sleep_data_for_date":
                    output = json.dumps(get_sleep_data_for_date(**args))
                elif name == "fetch_and_parse_notion_entries":
                    output = json.dumps(fetch_and_parse_notion_entries(**args))
                else:
                    output = f"No local handler implemented for {name}"

                outputs.append({
                    "tool_call_id": call.id,
                    "output": output
                })

            # Submit tool outputs back to assistant
            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=outputs
            )

        elif run_status.status == "completed":
            break
        elif run_status.status == "failed":
            raise Exception("Assistant run failed")

    # Once run is completed, fetch and parse the latest assistant response
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    latest_message_text = None
    image_data_list = []

    for content_block in messages.data[0].content:
        if content_block.type == "text":
            latest_message_text = content_block.text.value.strip()
        elif content_block.type == "image_file":
            image_file_id = content_block.image_file.file_id
            image_content = client.files.content(file_id=image_file_id)
            image_bytes = b"".join(image_content.iter_bytes())
            image_data_list.append(image_bytes)

    # If nothing was returned, show fallback message
    if latest_message_text is None and not image_data_list:
        return {
            "text": "(No content returned.)",
            "images": []
        }

    return {
        "text": latest_message_text,
        "images": image_data_list
    }




# Launching AURA & Create a New Assistant

> Run the cell below **every time you want to start a new assistant**.

This cell:
- Creates a new assistant
- Sets up a persistent thread
- Primes the assistant using the file search tool (run twice for better context retention)


In [39]:
 # Create  assistant ( needs to be done each time a new conversation is starting)

assistant = create_new_assistant("April_2025_Journal_Entries.txt")

# Create a persistent thread
thread = client.beta.threads.create()

# Repeated twice for solid priming
prime_thread_with_file_search(thread, assistant.id)
prime_thread_with_file_search(thread, assistant.id)



# Interactive Chat Interface

This widget lets you talk to the assistant and see responses in real time.

### Example Prompts to Try:
- `"what was my mood and head-space at based on my journal entries from the 14th to 20th of April 2025"`
- `"How was my sleep and productivity on the 19th of April 2025."`
- `"Was there any pattern between deep work and sleep quality form the 14th to the 20th of April? Find any correlations and plot the relationship"`
- `"Compare my recovery scores to my productivity form the 14th to the 20th of April. Use a visualisation"`


>  **Important Design Note:**  
> The assistant can sometimes ( not always) be overloaded if asked to use all the tools together for a long period of time. To prevent confusion or overload, the user is expected to use a few prompts to extract all the information and analysis for a time period.


> **Re-running Assistant Note**:  
> if you want to start a new conversation, run the cell above and create a new assistant and thread, and rerun the cell below.

> If you get an error for whatever reason you'll need to rerun the assistant


In [35]:
# Output widget to display full conversation history
chat_history = widgets.Output()

# Text input and send button
input_box = widgets.Textarea(
    placeholder='Ask a question or follow up...',
    description='User:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)
submit_button = widgets.Button(description="Send", button_style='info')

# Callback for the button
def on_submit(b):
    user_input = input_box.value.strip()
    if user_input == "":
        return

    # Disable the button and show waiting status
    submit_button.disabled = True
    submit_button.description = "Waiting..."

    # Display user message and placeholder
    with chat_history:
        display(Markdown(f"**User:**\n\n{user_input}"))
        display(Markdown("**Assistant:** _(thinking...)_"))

    try:
        response = interact_with_assistant(assistant, thread, user_input)
        if isinstance(response, str):
            # fallback in case response is incorrectly formatted
            response = {"text": response, "images": []}
    except Exception as e:
        response = {"text": f"(Error: {str(e)})", "images": []}

    # Display response content
    with chat_history:
        if response.get("text"):
            display(Markdown(f"**Assistant:**\n\n{response['text']}"))
        for img_bytes in response.get("images", []):
            display(Image(data=img_bytes, format='PNG'))

    # Reset
    input_box.value = ""
    submit_button.disabled = False
    submit_button.description = "Send"

# Connect and display
submit_button.on_click(on_submit)
display(chat_history)
display(input_box, submit_button)

Output()

Textarea(value='', description='User:', layout=Layout(height='150px', width='100%'), placeholder='Ask a questi…

Button(button_style='info', description='Send', style=ButtonStyle())

# Previous Worked Examples

### Example 1

In [ ]:
# Output widget to display full conversation history
chat_history = widgets.Output()

# Text input and send button
input_box = widgets.Textarea(
    placeholder='Ask a question or follow up...',
    description='User:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)
submit_button = widgets.Button(description="Send", button_style='info')

# Callback for the button
def on_submit(b):
    user_input = input_box.value.strip()
    if user_input == "":
        return

    # Disable the button and show waiting status
    submit_button.disabled = True
    submit_button.description = "Waiting..."

    # Display user message and placeholder
    with chat_history:
        display(Markdown(f"**User:**\n\n{user_input}"))
        display(Markdown("**Assistant:** _(thinking...)_"))

    try:
        response = interact_with_assistant(assistant, thread, user_input)
        if isinstance(response, str):
            # fallback in case response is incorrectly formatted
            response = {"text": response, "images": []}
    except Exception as e:
        response = {"text": f"(Error: {str(e)})", "images": []}

    # Display response content
    with chat_history:
        if response.get("text"):
            display(Markdown(f"**Assistant:**\n\n{response['text']}"))
        for img_bytes in response.get("images", []):
            display(Image(data=img_bytes, format='PNG'))

    # Reset
    input_box.value = ""
    submit_button.disabled = False
    submit_button.description = "Send"

# Connect and display
submit_button.on_click(on_submit)
display(chat_history)
display(input_box, submit_button)

Output()

Textarea(value='', description='User:', layout=Layout(height='150px', width='100%'), placeholder='Ask a questi…

Button(button_style='info', description='Send', style=ButtonStyle())

### Example 2

In [ ]:
# Output widget to display full conversation history
chat_history = widgets.Output()

# Text input and send button
input_box = widgets.Textarea(
    placeholder='Ask a question or follow up...',
    description='User:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)
submit_button = widgets.Button(description="Send", button_style='info')

# Callback for the button
def on_submit(b):
    user_input = input_box.value.strip()
    if user_input == "":
        return

    # Disable the button and show waiting status
    submit_button.disabled = True
    submit_button.description = "Waiting..."

    # Display user message and placeholder
    with chat_history:
        display(Markdown(f"**User:**\n\n{user_input}"))
        display(Markdown("**Assistant:** _(thinking...)_"))

    try:
        response = interact_with_assistant(assistant, thread, user_input)
        if isinstance(response, str):
            # fallback in case response is incorrectly formatted
            response = {"text": response, "images": []}
    except Exception as e:
        response = {"text": f"(Error: {str(e)})", "images": []}

    # Display response content
    with chat_history:
        if response.get("text"):
            display(Markdown(f"**Assistant:**\n\n{response['text']}"))
        for img_bytes in response.get("images", []):
            display(Image(data=img_bytes, format='PNG'))

    # Reset
    input_box.value = ""
    submit_button.disabled = False
    submit_button.description = "Send"

# Connect and display
submit_button.on_click(on_submit)
display(chat_history)
display(input_box, submit_button)


Output()

Textarea(value='', description='User:', layout=Layout(height='150px', width='100%'), placeholder='Ask a questi…

Button(button_style='info', description='Send', style=ButtonStyle())

### Example 3

In [41]:
# Output widget to display full conversation history
chat_history = widgets.Output()

# Text input and send button
input_box = widgets.Textarea(
    placeholder='Ask a question or follow up...',
    description='User:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)
submit_button = widgets.Button(description="Send", button_style='info')

# Callback for the button
def on_submit(b):
    user_input = input_box.value.strip()
    if user_input == "":
        return

    # Disable the button and show waiting status
    submit_button.disabled = True
    submit_button.description = "Waiting..."

    # Display user message and placeholder
    with chat_history:
        display(Markdown(f"**User:**\n\n{user_input}"))
        display(Markdown("**Assistant:** _(thinking...)_"))

    try:
        response = interact_with_assistant(assistant, thread, user_input)
        if isinstance(response, str):
            # fallback in case response is incorrectly formatted
            response = {"text": response, "images": []}
    except Exception as e:
        response = {"text": f"(Error: {str(e)})", "images": []}

    # Display response content
    with chat_history:
        if response.get("text"):
            display(Markdown(f"**Assistant:**\n\n{response['text']}"))
        for img_bytes in response.get("images", []):
            display(Image(data=img_bytes, format='PNG'))

    # Reset
    input_box.value = ""
    submit_button.disabled = False
    submit_button.description = "Send"

# Connect and display
submit_button.on_click(on_submit)
display(chat_history)
display(input_box, submit_button)

Output()

Textarea(value='', description='User:', layout=Layout(height='150px', width='100%'), placeholder='Ask a questi…

Button(button_style='info', description='Send', style=ButtonStyle())